In [1]:
import os
from utils.loader import load_data, unicorn_fs, convert_to_mne, unicorn_eeg_channels
from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs
import matplotlib.pyplot as plt
from utils.save_csv import save_csv

unicorn_channels = ["Fz", "C3", "Cz", "C4", "Pz", "PO7", "Oz", "PO8", "acc_x", "acc_y", "acc_z"]

In [2]:
path_data = 'data/recordings/playing/'
files = [f for f in os.listdir(path_data) if f.endswith('.csv')]
path_file = os.path.join(path_data, files[0])

eeg, trigger, dataframe = load_data(path_file, header=False, fs=unicorn_fs, skiprows=5, names = unicorn_channels)
raw_data = convert_to_mne(eeg, trigger, fs=unicorn_fs, chs=unicorn_eeg_channels, recompute=False) 

# Filtering
filtered = raw_data.copy() 
filtered.notch_filter(50) 
filtered.notch_filter(60) 
filtered.filter(1,40)

Creating RawArray with float64 data, n_channels=8, n_times=5141
    Range : 0 ... 5140 =      0.000 ...    20.560 secs
Ready.
Creating RawArray with float64 data, n_channels=1, n_times=5141
    Range : 0 ... 5140 =      0.000 ...    20.560 secs
Ready.


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1651 samples (6.604 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband e

Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,11 points
Good channels,"8 EEG, 1 Stimulus"
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,250.00 Hz
Highpass,1.00 Hz
Lowpass,40.00 Hz


In [3]:
n_components = 8

ica = ICA(n_components=n_components, max_iter="auto", random_state=97)
ica.fit(filtered)

Fitting ICA to data using 8 channels (please be patient, this may take a while)
Selecting by number: 8 components


Fitting ICA took 0.3s.


Method,fastica
Fit parameters,algorithm=parallelfun=logcoshfun_args=Nonemax_iter=1000
Fit,20 iterations on raw data (5141 samples)
ICA components,8
Available PCA components,8
Channel types,eeg
ICA components marked for exclusion,—


In [4]:
explained_var_ratio = ica.get_explained_variance_ratio(filtered)

for channel_type, ratio in explained_var_ratio.items():
    print(f"Fraction of {channel_type} variance explained by all components: " f"{ratio}")

    for component in range(n_components):
        explained_var_ratio = ica.get_explained_variance_ratio(filtered, components=[component], ch_type=channel_type)
        ratio_percent = round(100 * explained_var_ratio[channel_type])
        print(
            f"Fraction of variance in {channel_type} signal explained by component {component}: "
            f"{ratio_percent}%"
        )

Fraction of eeg variance explained by all components: 1.0
Fraction of variance in eeg signal explained by component 0: 61%
Fraction of variance in eeg signal explained by component 1: 33%
Fraction of variance in eeg signal explained by component 2: 0%
Fraction of variance in eeg signal explained by component 3: 3%
Fraction of variance in eeg signal explained by component 4: 1%
Fraction of variance in eeg signal explained by component 5: 1%
Fraction of variance in eeg signal explained by component 6: 0%
Fraction of variance in eeg signal explained by component 7: 1%


In [ ]:
raw_data.load_data()
ica.plot_sources(raw_data, show_scrollbars=False)
ica.plot_overlay(raw_data, exclude=[0], picks="eeg")

In [ ]:
ica.exclude = [0, 1, 2, 3, 4, 5, 7]  # indices chosen based on various plots above
reconst_raw = raw_data.copy()
ica.apply(reconst_raw)

reconst_raw.plot(scalings=1e-4)

save_csv('ica.csv', reconst_raw)
print(reconst_raw)

# pxx_diff = reconst_raw.compute_psd(fmin=0, fmax=50)
# fig = pxx_diff.plot()
# plt.show()
# plt.close()